In [ ]:
# EXAMPLE 2 - Create linear regression models for all 60 pole transformers.
# Read PI data via AF SDK.

import sys
sys.path.append('C:\\Program Files (x86)\\PIPC\\AF\\PublicAssemblies\\4.0\\')

import clr  # 'clr" module provides access to the .net Common Language Runtime. From 'pythonnet' package.
clr.AddReference('OSIsoft.AFSDK')  

# Access the components of the AF SDK.
from OSIsoft.AF import *
from OSIsoft.AF.PI import *  
from OSIsoft.AF.Search import *  
from OSIsoft.AF.Asset import *  
from OSIsoft.AF.Data import *  
from OSIsoft.AF.Time import * 

# Import Python packages.
import adodbapi as ado                        # Support for accessing MS SQL to store model coefficients.
import pandas as pd                           # Dataframe support.

from sklearn.linear_model import LinearRegression # Linear regression model from the scikit-learn package.

In [ ]:
# Establish PI AF access.
afServers = PISystems()

# Connect to AF server by name.
afServer = afServers['PISRV01']

# Select AF database for this analysis.
afDatabase = DB = afServer.Databases.get_Item("Distribution Network")  

# Search AF Heirarchy for 'Single Phase Transformer' elements. 
afTemplate = "Template:'Single Phase Transformer'"

# Search for list of pole transformer elements in the AF model.
afSearch = AFElementSearch(afDatabase, "Mysearch", afTemplate)
print("Search AF heirarchy for elements with following characteristics. \n", afTemplate)

# Print the number of transformers fondunder this branch of the AF Heirarchy.
print('No. elements found:', afSearch.GetTotalCount())

In [ ]:
# Define time range variable for extracted data.
timeRange = AFTimeRange()
timeRange.StartTime = AFTime('15-Jun-17')
timeRange.EndTime = AFTime('31-Aug-17')
print('DataFrame time range:', timeRange, '\n')

# Calculate the number of hours in the time range so we can get interpolated values returned.
hrIntervals = int((-1)*((timeRange.EndTime.UtcSeconds - timeRange.StartTime.UtcSeconds)/3600 + 1))
print('Number of hours in time range (negative value used for getting interpoated values SDK call):', hrIntervals)

In [ ]:
# Create empty pandas dataframe for transformer data.
columns =['Transformer', 'TimeStamp', 'Temperature', 'Humidity','Wind','Wh Load', 'Wh Load-14d', 'Wh Load-7d']
modellingData = pd.DataFrame(columns=columns)

# Create empty list to put the transformer names..
transformerNames = []

# Loop through Transformer elements.
for transformerElement in afSearch.FindElements():
    
    # Add this transformer to the transformerNames list.
    transformerNames.append(transformerElement.Name)

    # Initialize/reset lists to store unpacked data.
    timestamp = []; asset=[]
    temperature = []; humidity= []; wind = []
    load = [];  load_7d = []; load_14d = [] 

    # Get needed transformer attributes.
    attTemperature = transformerElement.Attributes.get_Item("Ambient Temperature")
    attHumidity = transformerElement.Attributes.get_Item("Relative Humidity")
    attWind = transformerElement.Attributes.get_Item("Wind Speed")
    attLoad = transformerElement.Attributes.get_Item("Wh Delivered Load")
    attLoad_7d = transformerElement.Attributes.get_Item("Wh Delivered Load - 7d")
    attLoad_14d = transformerElement.Attributes.get_Item("Wh Delivered Load - 14d")

    # Populate "Transformer", "Timestamp" and "Temperature" value lists.
    for item in attTemperature.GetValues(timeRange,hrIntervals,attTemperature.DefaultUOM):
        # Transformer name.
        asset.append(transformerElement.Name)
        # Timestamp.
        timestamp.append(item.Timestamp.ToString())
        # Temperature
        temperature.append(item.Value)

    # Populate "Humidity" value lists.    
    for item in attHumidity.GetValues(timeRange,hrIntervals,attHumidity.DefaultUOM):
        humidity.append(item.Value)

    # Populate "Wind" value lists.    
    for item in attWind.GetValues(timeRange,hrIntervals,attWind.DefaultUOM):
        wind.append(item.Value)

    # Populate "Wh Load" value lists.    
    for item in attLoad.GetValues(timeRange,hrIntervals,attLoad.DefaultUOM):
        load.append(item.Value)

    # Populate "Wh Load -7d" value lists.    
    for item in attLoad_7d.GetValues(timeRange,hrIntervals,attLoad_7d.DefaultUOM):
        load_7d.append(item.Value)

    # Populate "Wh Load - 14d" value lists.    
    for item in attLoad_14d.GetValues(timeRange,hrIntervals,attLoad_14d.DefaultUOM):
        load_14d.append(item.Value)

    # Put lists of data for this transformer into a temporary dataframe
    thisTransformerData = pd.DataFrame({'Transformer':asset,'TimeStamp':timestamp,
                                        'Temperature':temperature,'Humidity':humidity,'Wind':wind,
                                        'Wh Load':load,'Wh Load-7d':load_7d,'Wh Load-14d':load_14d})

    # Add this transformer's data to the others.
    modellingData = pd.concat([modellingData, thisTransformerData],ignore_index=True)

# Reindex or change the order of columns
#columnsTitles = ['Transformer','TimeStamp','Temperature','Humidity','Wind','Wh Load','Wh Load-7d','Wh Load-14d']
#modellingData = xPhaseTransformerData.reindex(columns=columnsTitles)
    
modellingData.tail()

In [ ]:
# Check the list of Transformers to make sure there are 60 of them.
print(transformerNames)

In [ ]:
# In order to analyze transformers individually, we need to set the datframe's index to the "Transformer" column. 
modellingData = modellingData.set_index("Transformer", drop=False)

# Take a look, see the difference?
modellingData.head()

In [ ]:
# Using the "adodbapi" package, to connect to the "Predictive Equations" MS SQL.

# Set connection parameters.
con_string = 'DRIVER={SQL Server};SERVER=PISRV01;DATABASE=PIWorld;Trusted_Connection=Yes;'

# Connect to "Distribution Network Lab" model in AF through the PI SQL Client.
SQL_connection = ado.connect(con_string)

# Create a cursor object to access the data serer for the "Distribution Network Lab" database in AF.
SQL_cursor = SQL_connection.cursor()

# Create linear regression object from the "sklearn" package we imported earlier.
LinReg = LinearRegression()

# Looping through the trasformer list, Perform linear regression on each transformer.
for transformer in transformerNames:
    
    # Create dataframe for one transformer.
    transformerData = modellingData.loc[transformer,:]
    
    # Perform linear regression fit
    LinReg.fit(transformerData[["Wh Load-7d","Wh Load-14d","Temperature","Humidity"]],transformerData["Wh Load"])
    
    # Update asset ID value with the name of this transformer.
    asset_id = "'"+transformer+"'"
    
    # Print equation.
    print(transformer, "Eq:\n", LinReg.coef_[0], "*","'Wh Delivered Load - 7d' + ",LinReg.coef_[1], "*","'Wh Delivered Load - 14d' + ",
    LinReg.coef_[2], "*", "'Ambient Temperature' +", LinReg.coef_[3], "*","'Relative Humidity' +(",LinReg.intercept_, ")" )    
    
    ## INSERT and UPDATE queries to load table for the first time or update an existing one.
    # Construct query to add this transfomer's model coefficients.
    insert_query = f'INSERT [Predictive Equations] ([Asset ID], Coefficient_0, Coefficient_1, Coefficient_2, Coefficient_3, Coefficient_4, Intercept) values({asset_id}, {LinReg.coef_[0]}, {LinReg.coef_[1]}, {LinReg.coef_[2]}, {LinReg.coef_[3]}, null, {LinReg.intercept_})'

    update_query = f'UPDATE [Predictive Equations] SET Coefficient_0={LinReg.coef_[0]}, Coefficient_1={LinReg.coef_[1]},Coefficient_2={LinReg.coef_[2]},Coefficient_3={LinReg.coef_[3]},Intercept={LinReg.intercept_} WHERE [Asset ID]= {asset_id}'
    
    # Insert this record into the "Predictive Equations" table.
    SQL_cursor.execute(update_query)
    
# Commit the queries to write the data into SQL.
SQL_connection.commit()

# Close database connections.
SQL_connection.close()